In [19]:
import torch
import torch.nn.functional as F
import sys
import os


sys.path.append('C:\\Users\\raghu\\test\\DNSSgit\\DNN-based_source_separation\\src')


from utils.filterbank import choose_filterbank
from utils.streaming_filterbank import choose_filterbank as choose_streaming_filterbank
from modules.norm import CumulativeLayerNorm1d 
from modules.streaming_norm import CumulativeLayerNorm1d as StreamingCumulativeLayerNorm1d 

# now we can import the module in the parent
# directory.
#from src.models.streaming_encoder import Encoder





In [22]:
n_basis=512
kernel_size=128       
chunk_size=kernel_size #Needs to be a multiple of kernel size
stride=64
enc_basis='trainable'
dec_basis='trainable'
n_sources=2
encoder, decoder = choose_filterbank(n_basis, kernel_size=kernel_size, stride=stride, enc_basis=enc_basis, 
                                     dec_basis=dec_basis, enc_nonlinear='relu')
streaming_encoder, streaming_decoder = choose_streaming_filterbank(n_basis, kernel_size=kernel_size, stride=stride, enc_basis=enc_basis, 
                                     dec_basis=dec_basis, enc_nonlinear='relu')
norm1d = CumulativeLayerNorm1d(n_basis)
streaming_norm1d = StreamingCumulativeLayerNorm1d(n_basis)

# Now make that weights and bias for streaming_encoder and decoder are identical to the non streaming one.
streaming_encoder.load_state_dict(encoder.state_dict())
streaming_decoder.load_state_dict(decoder.state_dict())
streaming_norm1d.load_state_dict(norm1d.state_dict())
print(streaming_encoder)

Creating streaming encoder
-64
Encoder(
  (conv1d): StreamingConv1d(1, 512, kernel_size=(128,), stride=(64,), bias=False)
  (nonlinear1d): ReLU()
)


In [23]:
        """
        Args:
            input (batch_size, C_in, T)
        Returns:
            output (batch_size, n_sources, T) or (batch_size, n_sources, C_in, T)
            latent (batch_size, n_sources, n_basis, T'), where T' = (T-K)//S+1
        """
        batch_size=1
        
        input = torch.randn(batch_size,1,1920)

        n_dims = input.dim()

        if n_dims == 3:
            batch_size, C_in, T = input.size()
            assert C_in == 1, "input.size() is expected (?, 1, ?), but given {}".format(input.size())
        elif n_dims == 4:
            batch_size, C_in, n_mics, T = input.size()
            assert C_in == 1, "input.size() is expected (?, 1, ?, ?), but given {}".format(input.size())
            input = input.view(batch_size, n_mics, T)
        else:
            raise ValueError("Not support {} dimension input".format(n_dims))

        padding = (stride - (T - kernel_size) % stride) % stride
        padding_left = padding // 2
        padding_right = padding - padding_left

        input = F.pad(input, (padding_left, padding_right))
        w = encoder(input)
       # w = norm1d(w)
        w_ref =  w.detach().clone()
        w = w.unsqueeze(dim=1)
        mask= torch.ones(batch_size,n_sources,w.size()[2], w.size()[3])
    
        print('pre-mask w',w.size())
        w=w*mask
        w = w.view(batch_size*n_sources, n_basis, -1)
        print('Decoder input shape', w.size())
        x = decoder(w)
        # Split input into two parts and check if things work
        w_total=torch.Tensor([])
        x_total=torch.Tensor([])
        num_chunks = input.size()[2]//chunk_size+1
        print('number of chunks', num_chunks)
        for i in range(num_chunks):
            start_idx = i*chunk_size
            end_idx = min((i+1)*chunk_size, input.size()[2])
            input_slice = input[:,:,start_idx:end_idx]
            w_s = streaming_encoder(input_slice)
           # print('w_s.size',w_s.size())
            if w_s is not None:
             #   w_s = streaming_norm1d(w_s)
                w_total=torch.cat([w_total,w_s], dim=2)
                w_s = w_s.unsqueeze(dim=1)
                streaming_mask = torch.ones(batch_size,n_sources,w_s.size()[2], w_s.size()[3])
    
                w_s=w_s*streaming_mask
                w_s = w_s.view(batch_size*n_sources, n_basis, -1)
                x_hat = streaming_decoder(w_s)
                x_total=torch.cat([x_total,x_hat], dim=2)
   #     print(w_ref.size(), w_total.size())
   #     print(w_ref)
   #     print(w_total)
        print('Ref decoder output size',x.size()) 
        print(x_total.size())
        # print(x)
       # print(x_total)
        print(torch.norm(x[:,:,:x_total.size()[2]]-x_total))
      #  print(w[0,:,0])

       # print(w_total[0,:,0])
        print('Encoder output', torch.norm(w-w_total))
  
        if n_dims == 3:
            x_hat = x_hat.view(batch_size, n_sources, -1)
        else: # n_dims == 4
            x_hat = x_hat.view(batch_size, n_sources, 1, -1)
        print('reshaped decoder output', x_hat.size())
        output = F.pad(x_hat, (-padding_left, -padding_right))
        print('Final output size', output.size())
    


pre-mask w torch.Size([1, 1, 512, 29])
Decoder input shape torch.Size([2, 512, 29])
number of chunks 16
Streaming Conv: input size torch.Size([1, 1, 128])
Decoder:Input before stat padding torch.Size([2, 512, 1])
First output torch.Size([2, 1, 128])
Decoder output torch.Size([2, 1, 64])
Streaming Conv: State size torch.Size([1, 1, 64])
Streaming Conv: input size torch.Size([1, 1, 128])
Decoder:Input before stat padding torch.Size([2, 512, 2])
Decoder output torch.Size([2, 1, 128])
Streaming Conv: State size torch.Size([1, 1, 64])
Streaming Conv: input size torch.Size([1, 1, 128])
Decoder:Input before stat padding torch.Size([2, 512, 2])
Decoder output torch.Size([2, 1, 128])
Streaming Conv: State size torch.Size([1, 1, 64])
Streaming Conv: input size torch.Size([1, 1, 128])
Decoder:Input before stat padding torch.Size([2, 512, 2])
Decoder output torch.Size([2, 1, 128])
Streaming Conv: State size torch.Size([1, 1, 64])
Streaming Conv: input size torch.Size([1, 1, 128])
Decoder:Input bef

In [16]:
x = torch.randn(1,2,12)
stride=2
print(decoder.kernel_size, decoder.stride)
myMod = torch.nn.ConvTranspose1d(2,1,kernel_size=4,stride=2, bias=False)
print(myMod.weight)
z = myMod(x)
print(x)
print(z)
print(z.size())

x1 = x[:,:,0:5]
z1 = myMod(x1)
print(z1)
x2 = x[:,:,4:12]
z2=myMod(x2)
print(z2)
z_out = torch.cat([z1[:,:,0:-stride], z2[:,:,stride:]], dim=2)
print(z_out)

16 8
Parameter containing:
tensor([[[ 0.0837,  0.1937,  0.1692, -0.1434]],

        [[-0.4218, -0.1416,  0.4376, -0.3596]]], requires_grad=True)
tensor([[[ 0.7244,  0.1181,  1.8424, -1.3256, -0.5561,  0.9746,  0.8712,
          -0.4667, -1.1989,  1.0680,  0.6017,  0.4766],
         [ 0.9860,  1.3744,  1.7378, -1.1001, -1.4678, -0.0912,  0.6194,
          -0.5583, -0.8538, -1.4674,  0.7687,  0.7143]]])
tensor([[[-3.5526e-01,  6.9052e-04, -1.5800e-02, -6.3019e-01,  4.2620e-02,
          -4.0037e-01,  1.4253e+00, -9.9015e-01, -1.3312e-01,  6.8589e-01,
          -6.1642e-01,  8.0929e-01, -6.3340e-02, -2.5957e-02,  6.1489e-01,
          -3.5905e-01, -6.3487e-02,  1.5637e-01,  1.3186e-01,  8.9365e-01,
          -7.3532e-01,  3.8215e-01,  1.7681e-01, -3.7154e-01,  3.9324e-01,
          -3.2522e-01]]], grad_fn=<ConvolutionBackward0>)
torch.Size([1, 1, 26])
tensor([[[-3.5526e-01,  6.9052e-04, -1.5800e-02, -6.3019e-01,  4.2620e-02,
          -4.0037e-01,  1.4253e+00, -9.9015e-01, -1.3312e-01,  6

In [ ]:
wd=myMod.weight
print(wd.size())
# Getting the outputs
print(wd[0,0,0]*x[0,0,0]+wd[1,0,0]*x[0,1,0])
print(wd[0,0,1]*x[0,0,0]+wd[1,0,1]*x[0,1,0] )
print(wd[0,0,2]*x[0,0,0]+wd[1,0,2]*x[0,1,0]+wd[0,0,0]*x[0,0,1]+wd[1,0,0]*x[0,1,1] )
print(wd[0,0,3]*x[0,0,0]+wd[1,0,3]*x[0,1,0]+wd[0,0,1]*x[0,0,1]+wd[1,0,1]*x[0,1,1] )


In [ ]:
    from models.tdcn import TimeDilatedConvNet
    batch_size = 1
    T = 40
    in_channels, out_channels, skip_channels = 2, 2, 2
    kernel_size = 3
    num_blocks = 1
    num_layers = 4
    dilated, separable = True, True
    causal = True
    nonlinear = 'prelu'
    norm = False

    input = torch.randn((batch_size, in_channels, T), dtype=torch.float)

    model = TimeDilatedConvNet(in_channels, hidden_channels=out_channels, skip_channels=skip_channels, kernel_size=kernel_size, num_blocks=num_blocks, num_layers=num_layers, dilated=dilated, separable=separable, causal=causal, nonlinear=nonlinear, norm=norm)

    output = model(input)

    print(input.size())
   # print(input)
    print(output.size())
    print(output)
    
# MAgic numbers for num_layer=2 (2+4=6)
# For num_layers=3 (2+4+8 =14)
# For num_layers=4 (2+4+8+16 =26)
    input1 = input[:,:,:30]
    input2 = input[:,:,4:]
    
    output1 = model(input1)
    output2 = model(input2)
    print(output1)
    print(output2)

In [ ]:
import torch.nn as nn

class StreamingConv1d(nn.Conv1d):
    def __init__(self,in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None):
        super().__init__(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias, padding_mode, device, dtype)
        
        self.state=None
        self.state_len = self.kernel_size[0]*self.dilation[0]-self.stride[0]*self.dilation[0]
        print(-self.kernel_size[0]*self.dilation[0]+self.stride[0])
    def forward(self, input):
        # input is (1,kL)
        # output is (1,k,N)
      #  print('Encoder:State',self.state.size())
      #  print('Encoder:Input before stat padding',input.size())
        x=None
 #       print('input size',input.size())
        if self.state is None:
            sz_size=0
        else:
            sz_size=self.state.size()[2]
            print('Streaming Conv: State size',self.state.size())
            
        if sz_size+input.size()[2] >(self.kernel_size[0]-1)*self.dilation[0]:
            print('Streaming Conv: input size',input.size())
            if sz_size > 0:
                extended_input = torch.cat([self.state, input], dim=2)
            else:
                extended_input = input
            x = super().forward(extended_input)
         #  print('Encoder:state+input dim', input.size()) 
        # What to do if the input is too short?
        # Add it to state, return empty!
        
        if sz_size > 0:
            self.state = torch.cat([self.state, input[:,:,-(self.kernel_size[0]-1)*self.dilation[0]:]], dim=2)
        else:
            self.state =  input[:,:,-(self.kernel_size[0]-1)*self.dilation[0]:]
            
# Keep the needed elements
 #       print("State size, before pruning", self.state.size())
        self.state = self.state[:,:,-(self.kernel_size[0]-1)*self.dilation[0]:]
 #       print("State size, after pruning", self.state.size())
        return x

In [ ]:
batch_size=1
out_channels=2
kernel_size=3
stride=2
padding=0
dilation=4
bias=False

input = torch.randn(batch_size,1,30)

m1 = nn.Conv1d(1,out_channels, kernel_size,stride,padding,dilation,1,bias)
m2 = StreamingConv1d(1,out_channels, kernel_size,stride,padding,dilation,1,bias)
m2.weight=m1.weight

ref = m1(input)


In [ ]:
chunk_size=8
out_total=torch.Tensor([])
num_chunks = input.size()[2]//chunk_size+1
print('number of chunks', num_chunks)
for i in range(num_chunks):
    start_idx = i*chunk_size
    end_idx = min((i+1)*chunk_size, input.size()[2])
    input_slice = input[:,:,start_idx:end_idx]
    out_slice =m2(input_slice)
#    print('Input slice', input_slice.size())
    if out_slice is not None:
        print('Out slice', out_slice.size())
        out_total=torch.cat([out_total,out_slice], dim=2)
        print('Out_total', out_total.size())
print(ref.size(), out_total.size())
print(ref)
print(out_total)
print(torch.norm(out_total-ref))



In [ ]:
from utils.tasnet import choose_layer_norm
class StreamingDepthwiseSeparableConv1d(nn.Module):
    def __init__(self, in_channels, out_channels=256, skip_channels=256, kernel_size=3, stride=2, dilation=1, causal=True, nonlinear=None, norm=True,use_batch_norm=False, dual_head=True, eps=1e-5):
        super().__init__()
        self.dual_head = dual_head
        self.norm = norm
        self.eps = eps

        self.depthwise_conv1d = StreamingConv1d(in_channels, in_channels, kernel_size=kernel_size, stride=stride, dilation=dilation, groups=in_channels)

        if nonlinear is not None:
            if nonlinear == 'prelu':
                self.nonlinear1d = nn.PReLU()
            else:
                raise ValueError("Not support {}".format(nonlinear))
            self.nonlinear = True
        else:
            self.nonlinear = False

        if norm:
             if use_batch_norm:
                 norm_name='BN'
             else:
                 norm_name = 'cLN' if causal else 'gLN'
             self.norm1d = StreamingCumulativeLayerNorm1d(in_channels)
        if dual_head:
            self.output_pointwise_conv1d = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1)

        self.skip_pointwise_conv1d = nn.Conv1d(in_channels, skip_channels, kernel_size=1, stride=1)

    def forward(self, input):
        nonlinear, norm = self.nonlinear, self.norm
        dual_head = self.dual_head

        x = self.depthwise_conv1d(input)

        if nonlinear:
            x = self.nonlinear1d(x)
        if norm:
            if x is not None:
                print('before norm', x.size())
                x = self.norm1d(x)

        if x is None:
            return None,None

        if dual_head:
            output = self.output_pointwise_conv1d(x)
        else:
            output = None

        skip = self.skip_pointwise_conv1d(x)

        return output,skip


In [ ]:
class DepthwiseSeparableConv1d(nn.Module):
    def __init__(self, in_channels, out_channels=256, skip_channels=256, kernel_size=3, stride=2, dilation=1, causal=True, nonlinear=None, norm=True,use_batch_norm=False, dual_head=True, eps=1e-5):
        super().__init__()

        self.dual_head = dual_head
        self.norm = norm
        self.eps = eps

        self.depthwise_conv1d = nn.Conv1d(in_channels, in_channels, kernel_size=kernel_size, stride=stride, dilation=dilation, groups=in_channels)

        if nonlinear is not None:
            if nonlinear == 'prelu':
                self.nonlinear1d = nn.PReLU()
            else:
                raise ValueError("Not support {}".format(nonlinear))
            self.nonlinear = True
        else:
            self.nonlinear = False

        if norm:
             if use_batch_norm:
                 norm_name='BN'
             else:
                 norm_name = 'cLN' if causal else 'gLN'
             self.norm1d = CumulativeLayerNorm1d(in_channels)
        if dual_head:
            self.output_pointwise_conv1d = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1)

        self.skip_pointwise_conv1d = nn.Conv1d(in_channels, skip_channels, kernel_size=1, stride=1)

    def forward(self, input):
        nonlinear, norm = self.nonlinear, self.norm
        dual_head = self.dual_head

        x = self.depthwise_conv1d(input)

        if nonlinear:
            x = self.nonlinear1d(x)
        if norm:
            x = self.norm1d(x)

    

        if dual_head:
            output = self.output_pointwise_conv1d(x)
        else:
            output = None

        skip = self.skip_pointwise_conv1d(x)

        return output,skip


In [ ]:
#from models.tdcn import DepthwiseSeparableConv1d

ref_mod = DepthwiseSeparableConv1d(2,4,4,3,stride=1,dilation=64,causal=True, norm=True)
streaming_mod = StreamingDepthwiseSeparableConv1d(2,4,4,3,stride=1,dilation=64,causal=True, norm=True)

streaming_mod.load_state_dict(ref_mod.state_dict())

In [ ]:
import numpy as np
x = torch.randn(1,2,200)
y,skip=ref_mod(x)
chunk_size=8
out_total=torch.Tensor([])
skip_total=torch.Tensor([])
num_chunks = np.ceil(x.size()[2]/chunk_size).astype(np.int32)
print('number of chunks', num_chunks)
for i in range(num_chunks):
    start_idx = i*chunk_size
    end_idx = min((i+1)*chunk_size, x.size()[2])
    input_slice = x[:,:,start_idx:end_idx]
    print('Chunk Input slice', input_slice.size())
    
    out_slice, skip_slice =streaming_mod(input_slice)
    if out_slice is not None:
        print('Out slice', out_slice.size())
        out_total=torch.cat([out_total,out_slice], dim=2)
        print('Out_total', out_total.size())
    if skip_slice is not None:
        print('skip slice', skip_slice.size())
        skip_total=torch.cat([skip_total,skip_slice], dim=2)
        print('skip_total', skip_total.size())
print(torch.norm(y-out_total))
print(torch.norm(skip-skip_total))
print(y)
print(y.size())
print(out_total.size())
print(out_total)

In [ ]:
class StreamingResidualBlock1d(nn.Module):
    def __init__(self, num_features, hidden_channels=256, skip_channels=256, kernel_size=3, stride=2, dilation=1,
                 separable=False, causal=True, nonlinear=None, norm=True, use_batch_norm=False, dual_head=True, eps=1e-5):
        super().__init__()

        self.kernel_size, self.stride, self.dilation = kernel_size, stride, dilation
        self.separable, self.causal = separable, causal
        self.norm = norm
        self.dual_head = dual_head

        self.bottleneck_conv1d = nn.Conv1d(num_features, hidden_channels, kernel_size=1, stride=1)

        if nonlinear is not None:
            if nonlinear == 'prelu':
                self.nonlinear1d = nn.PReLU()
            else:
                raise ValueError("Not support {}".format(nonlinear))
            self.nonlinear = True
        else:
            self.nonlinear = False

        if norm:
            if use_batch_norm:
                norm_name='BN'
            else:
                norm_name = 'cLN' if causal else 'gLN'
            self.norm1d = StreamingCumulativeLayerNorm1d(hidden_channels)
            #choose_layer_norm(norm_name, hidden_channels, causal=causal, eps=eps)
        if separable:
            self.separable_conv1d = StreamingDepthwiseSeparableConv1d(hidden_channels, num_features, skip_channels=skip_channels, kernel_size=kernel_size, stride=stride, dilation=dilation, causal=causal, nonlinear=nonlinear, norm=norm, use_batch_norm=use_batch_norm, dual_head=dual_head, eps=eps)
        else:
            if dual_head:
                self.output_conv1d = nn.Conv1d(hidden_channels, num_features, kernel_size=kernel_size, dilation=dilation)
            self.skip_conv1d = nn.Conv1d(hidden_channels, skip_channels, kernel_size=kernel_size, dilation=dilation)
        self.first_time=True

    def forward(self, input):
        kernel_size, stride, dilation = self.kernel_size, self.stride, self.dilation
        nonlinear, norm = self.nonlinear, self.norm
        separable, causal = self.separable, self.causal
        dual_head = self.dual_head

        _, _, T_original = input.size()

        residual = input
        x = self.bottleneck_conv1d(input)

        if nonlinear:
            x = self.nonlinear1d(x)
        if norm:
            if x is not None:
                x = self.norm1d(x)
        
        print('T_original', T_original)
        padding = (T_original - 1) * stride - T_original + (kernel_size - 1) * dilation + 1
        print('padding', padding)
        if self.first_time:
            if causal:
                padding_left = padding
                padding_right = 0
            else:
                padding_left = padding//2
                padding_right = padding - padding_left
            self.first_time=False
            x = F.pad(x, (padding_left, padding_right))

        if separable:
            output, skip = self.separable_conv1d(x) # output may be None
        else:
            if dual_head:
                output = self.output_conv1d(x)
            else:
                output = None

            skip = self.skip_conv1d(x)
        if output is not None:
            print(output.size(), residual.size())
            output = output + residual
            return output, skip
        else:
            return None,skip

In [ ]:
from models.tdcn import ResidualBlock1d
ref_mod=ResidualBlock1d(2,4,4,3,stride=1,dilation=4,separable=True,norm=True)
streaming_mod=StreamingResidualBlock1d(2,4,4,3,stride=1,dilation=4,separable=True,norm=True)
streaming_mod.load_state_dict(ref_mod.state_dict())

In [ ]:
x = torch.randn(1,2,30)
y_ref, skip_ref=ref_mod(x)
chunk_size=20
out_total=torch.Tensor([])
skip_total =torch.Tensor([])
num_chunks = np.ceil(x.size()[2]/chunk_size).astype(np.int32)
print('number of chunks', num_chunks)
for i in range(num_chunks):
    start_idx = i*chunk_size
    end_idx = min((i+1)*chunk_size, x.size()[2])
    input_slice = x[:,:,start_idx:end_idx]
    print('Chunk Input slice', input_slice.size())
    
    out_slice, skip_slice =streaming_mod(input_slice)
    if out_slice is not None:
        print('Out slice', out_slice.size())
        out_total=torch.cat([out_total,out_slice], dim=2)
        print('Out_total', out_total.size())
    if skip_slice is not None:
        print('Skip slice', skip_slice.size())
        skip_total=torch.cat([skip_total,skip_slice], dim=2)
        print('Skip_total', skip_total.size())

print(torch.norm(y_ref-out_total))                             
print(y_ref.size())
print(out_total.size())
print(y_ref)
print(out_total)
      
print(skip_ref.size())
print(skip_total.size())
print(skip_ref)
print(skip_total)


In [ ]:
from models.conv_tasnet import ConvTasNet
ref = ConvTasNet(
        n_basis=32, kernel_size=16, stride=None, enc_basis='trainable', dec_basis='trainable',
        sep_hidden_channels=256, sep_bottleneck_channels=128, sep_skip_channels=128, sep_kernel_size=3, sep_num_blocks=3, sep_num_layers=8,
        dilated=True, separable=True,
        sep_nonlinear='prelu', sep_norm=True,use_batch_norm=False, mask_nonlinear='sigmoid',
        causal=True,
        n_sources=2,
        eps=1e-5,
        enc_nonlinear='relu'
    )

x= torch.randn(1,1,10000)
ref(x)

In [ ]:
EPS = 1e-12

class StreamingTimeDilatedConvNet(nn.Module):
    def __init__(self, num_features, hidden_channels=256, skip_channels=256, kernel_size=3, num_blocks=3, num_layers=10, dilated=True, separable=False, causal=True,use_batch_norm=False, nonlinear=None, norm=True, eps=EPS):
        super().__init__()

        self.num_blocks = num_blocks

        net = []

        for idx in range(num_blocks):
            if idx == num_blocks - 1:
                net.append(StreamingTimeDilatedConvBlock1d(num_features, hidden_channels=hidden_channels, skip_channels=skip_channels, kernel_size=kernel_size, num_layers=num_layers, dilated=dilated, separable=separable, causal=causal, nonlinear=nonlinear, norm=norm, use_batch_norm=use_batch_norm, dual_head=False, eps=eps))
            else:
                net.append(StreamingTimeDilatedConvBlock1d(num_features, hidden_channels=hidden_channels, skip_channels=skip_channels, kernel_size=kernel_size, num_layers=num_layers, dilated=dilated, separable=separable, causal=causal, nonlinear=nonlinear, norm=norm, use_batch_norm=use_batch_norm, dual_head=True, eps=eps))

        self.net = nn.Sequential(*net)

    def forward(self, input):
        num_blocks = self.num_blocks

        x = input
        skip_connection = 0

        for idx in range(num_blocks):
            x, skip = self.net[idx](x)
            x_size=0
            skip_size=0
            if x is not None:
                x_size=x.size()
            if skip is not None:
                skip_size=skip.size()
            print('Streaming TDCN', idx, x_size,skip_size)
            skip_connection = skip_connection + skip
            print('Streaming TDCN 2', idx, skip_connection.size())

        output = skip_connection

        return output

class StreamingTimeDilatedConvBlock1d(nn.Module):
    def __init__(self, num_features, hidden_channels=256, skip_channels=256, kernel_size=3, num_layers=10, dilated=True, separable=False, causal=True, nonlinear=None, norm=True, use_batch_norm=False, dual_head=True, eps=EPS):
        super().__init__()

        self.num_layers = num_layers

        net = []
        print('tdcn: Dilated:', dilated)
        for idx in range(num_layers):
            if dilated:
                dilation = 2**idx
                print('Dilation', idx, dilation)
                stride = 1
            else:
                dilation = 1
                stride = 2
            if not dual_head and idx == num_layers - 1:
                net.append(StreamingResidualBlock1d(num_features, hidden_channels=hidden_channels, skip_channels=skip_channels, kernel_size=kernel_size, stride=stride, dilation=dilation, separable=separable, causal=causal, nonlinear=nonlinear, norm=norm, use_batch_norm=use_batch_norm, dual_head=False, eps=eps))
            else:
                net.append(StreamingResidualBlock1d(num_features, hidden_channels=hidden_channels, skip_channels=skip_channels, kernel_size=kernel_size, stride=stride, dilation=dilation, separable=separable, causal=causal, nonlinear=nonlinear, norm=norm, use_batch_norm=use_batch_norm, dual_head=True, eps=eps))

        self.net = nn.Sequential(*net)

    def forward(self, input):
        num_layers = self.num_layers

        x = input
        skip_connection = 0

        for idx in range(num_layers):
            x, skip = self.net[idx](x)
            skip_connection = skip_connection + skip

        return x, skip_connection

In [ ]:
n_basis=32
kernel_size=16
stride=None
enc_basis='trainable'
dec_basis='trainable',
sep_hidden_channels=256
sep_bottleneck_channels=2
sep_skip_channels=2
sep_kernel_size=3
sep_num_blocks=3
sep_num_layers=8
dilated=True
separable=True
sep_nonlinear='prelu'
sep_norm=True
use_batch_norm=False
mask_nonlinear='sigmoid'
causal=True
n_sources=2
eps=1e-5
enc_nonlinear='relu'

ref_mod=TimeDilatedConvNet(
            num_features=sep_bottleneck_channels, hidden_channels=sep_hidden_channels, skip_channels=sep_skip_channels, 
            kernel_size=sep_kernel_size, num_blocks=sep_num_blocks, num_layers=sep_num_layers,
            dilated=dilated, separable=separable, causal=causal, nonlinear=sep_nonlinear, norm=True
        )
streaming_mod =StreamingTimeDilatedConvNet(
            num_features=sep_bottleneck_channels, hidden_channels=sep_hidden_channels, skip_channels=sep_skip_channels, 
            kernel_size=sep_kernel_size, num_blocks=sep_num_blocks, num_layers=sep_num_layers,
            dilated=dilated, separable=separable, causal=causal, nonlinear=sep_nonlinear, norm=True
        )

streaming_mod.load_state_dict(ref_mod.state_dict())

In [ ]:
x = torch.randn(1,2,40)
y1=ref_mod(x)
print('ref output',y1.size())
chunk_size=20
out_total=torch.Tensor([])
num_chunks = np.ceil(x.size()[2]/chunk_size).astype(np.int32)
print('number of chunks', num_chunks)
for i in range(num_chunks):
    start_idx = i*chunk_size
    end_idx = min((i+1)*chunk_size, x.size()[2])
    input_slice = x[:,:,start_idx:end_idx]
    print('Chunk Input slice', input_slice.size())
    
    out_slice =streaming_mod(input_slice)

    if out_slice is not None:
        print('Out slice', out_slice.size())
        out_total=torch.cat([out_total,out_slice], dim=2)
        print('Out_total', out_total.size())
print(y1)
print(y1.size())
print(out_total.size())
print(out_total)